In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from pose import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
reps = {}
main(reps)

In [ ]:
rep_no = list(range(1, reps['count']))
time = list(reps['time'].values())[:reps['count']-1]
correct_form = list(reps['correct_form'].values())[:reps['count']-1]
wrong_form = list(reps['wrong_form'].values())[:reps['count']-1]

In [ ]:
data = [rep_no, time, correct_form, wrong_form]
columns = ['rep_no', 'time', 'correct_form', 'wrong_form']

In [ ]:
data = pd.DataFrame(data, columns).transpose()

In [ ]:
data.head()

In [ ]:
figure = plt.figure(figsize=(10, 6))

sns.set_style('darkgrid', {'axes.edgecolor': 'black'})
sns.set_context('talk')
sns.lineplot(data=data, x='rep_no', y='time',
             color='darkviolet', lw=2,
            marker='o', ms=12, mec='white', mew=2)

plt.xticks(rep_no)
plt.xlabel('Rep no.')
plt.ylabel('Time (s)')
plt.title("Time spent doing each rep\n")

plt.show()

In [ ]:
sns.axes_style()

In [ ]:
figure2 = go.Figure()

figure2.add_trace(go.Scatter(x=data['rep_no'], y=data['time'],
                             marker=dict(size=12,
                                        line=dict(width=2, color='white')),
                         line=dict(color='darkviolet', width=2)))

figure2.update_layout(title='Time spent doing each rep',
                   xaxis_title='Rep no.',
                   yaxis_title='Time (s)',
                      xaxis=dict(dtick=1),
                    plot_bgcolor='#EAEAF2',
                     font_size=15)

figure2.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
figure2.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

figure2.show()